# ML-Session-3 Solution: Tuning the Machine Learning Model

This notebook provides complete solutions for the ML-session-3 exercises.
It demonstrates feature selection techniques and model validation strategies.

## 1. Setup and Library Imports

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import sklearn
from sklearn import preprocessing, model_selection
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

%matplotlib inline

# Set random seed for reproducibility
np.random.seed(42)

In [ ]:
# Helper function for model evaluation
def model_evaluate(model, test_F, test_L):
    test_L_pred = model.predict(test_F)
    print("Evaluation by using model:", type(model).__name__)
    print("Accuracy Score:", accuracy_score(test_L, test_L_pred))
    print("Confusion Matrix:")
    print(confusion_matrix(test_L, test_L_pred))
    return accuracy_score(test_L, test_L_pred)

## 2. Preprocessing Sherlock Dataset

In [ ]:
# Load and preprocess the dataset
df2 = pd.read_csv('sherlock/sherlock_mystery_2apps.csv')

# Remove irrelevant feature(s)
df2.drop('Unnamed: 0', axis=1, inplace=True)

# Remove rows with missing values
df2.dropna(inplace=True)

# Remove duplicate features
df2.drop('Mem', axis=1, inplace=True)

# Separate labels from features
df2_labels = df2['ApplicationName']
df2_features = df2.drop('ApplicationName', axis=1)

# Feature scaling
scaler = preprocessing.StandardScaler()
scaler.fit(df2_features)
df2_features_n = pd.DataFrame(scaler.transform(df2_features),
                              columns=df2_features.columns,
                              index=df2_features.index)

# Create a backup for later use
df2_features_n_backup = df2_features_n.copy()

print(f"Preprocessed dataset shape: {df2_features_n.shape}")
print(f"Features: {df2_features_n.columns.tolist()}")
print(f"\nFirst 10 rows:")
print(df2_features_n.head(10))

## 3. Feature Selection

### 3.1 Histogram Analysis

In [ ]:
# Create histogram panel for all features
plt.figure(figsize=(12.0, 9.0))
for (i, col) in enumerate(df2_features_n.columns):
    plt.subplot(4, 3, i+1)
    plt.hist(df2_features_n[col], bins=50, alpha=0.7, edgecolor='black')
    plt.title(col, fontweight='bold')
    plt.xlabel('Value')
    plt.ylabel('Frequency')

plt.subplots_adjust(top=0.92, bottom=0.08, left=0.10, right=0.95, hspace=0.75, wspace=0.35)
plt.suptitle('Distribution of All Features', fontsize=14, fontweight='bold')
plt.show()

print("Observation: Look for features with similar distributions - these may be duplicates.")

In [ ]:
# SOLUTION: Separate features by application for comparison
Apps = df2_labels.unique()
indx_app = {}
features_app = {}

for app in Apps:
    indx_app[app] = df2_labels[df2_labels == app].index
    features_app[app] = df2_features_n.loc[indx_app[app]]

print(f"Applications: {Apps}")
print(f"Facebook records: {len(features_app['Facebook'])}")
print(f"WhatsApp records: {len(features_app['WhatsApp'])}")

In [ ]:
# SOLUTION: Create histogram panel colored by application
plt.figure(figsize=(12.0, 9.0))
for (i, col) in enumerate(df2_features_n.columns):
    plt.subplot(4, 3, i+1)
    for app in Apps:
        plt.hist(features_app[app][col], bins=50, alpha=0.5, label=app)
    plt.title(col, fontweight='bold')
    plt.xlabel('Value')
    plt.ylabel('Frequency')
    if i == 0:
        plt.legend()

plt.subplots_adjust(top=0.92, bottom=0.08, left=0.10, right=0.95, hspace=0.75, wspace=0.35)
plt.suptitle('Feature Distributions by Application', fontsize=14, fontweight='bold')
plt.show()

print("\nObservation: Features with overlapping distributions between apps are less discriminative.")

### 3.2 Correlation Analysis

In [ ]:
# SOLUTION: Compute correlation matrix
df2_corr = df2_features_n.corr()
print("Correlation Matrix:")
print(df2_corr)
print("\nCorrelation values range from -1 (perfectly anti-correlated) to 1 (perfectly correlated)")

In [ ]:
# SOLUTION: Visualize correlation matrix as heatmap
plt.figure(figsize=(10.0, 10.0))
sns.heatmap(df2_corr, annot=True, fmt='.2f', vmax=1, vmin=-1, square=True, 
            cmap="RdBu_r", center=0, cbar_kws={"shrink": 0.8})
plt.title('Feature Correlation Matrix', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# SOLUTION: Identify highly correlated feature pairs
print("\nHighly Correlated Feature Pairs (|correlation| > 0.8):")
print("="*60)

# Find pairs with high correlation
high_corr_pairs = []
for i in range(len(df2_corr.columns)):
    for j in range(i+1, len(df2_corr.columns)):
        corr_val = df2_corr.iloc[i, j]
        if abs(corr_val) > 0.8:
            high_corr_pairs.append((
                df2_corr.columns[i],
                df2_corr.columns[j],
                corr_val
            ))
            print(f"{df2_corr.columns[i]:20s} <-> {df2_corr.columns[j]:20s}: {corr_val:7.4f}")

print("\nModerate Correlation Pairs (0.5 < |correlation| <= 0.8):")
print("="*60)
for i in range(len(df2_corr.columns)):
    for j in range(i+1, len(df2_corr.columns)):
        corr_val = df2_corr.iloc[i, j]
        if 0.5 < abs(corr_val) <= 0.8:
            print(f"{df2_corr.columns[i]:20s} <-> {df2_corr.columns[j]:20s}: {corr_val:7.4f}")

### 3.3 Remove Highly Correlated Features

In [ ]:
# SOLUTION: Drop highly correlated duplicate features
print("Removing highly correlated features: vsize, queue, guest_time")
df2_features_n.drop(['vsize', 'queue', 'guest_time'], axis=1, inplace=True)
print(f"Remaining features ({len(df2_features_n.columns)}): {df2_features_n.columns.tolist()}")

In [ ]:
# SOLUTION: Remove moderately correlated features
print("Removing moderately correlated features: utime, cminflt")
df2_features_n.drop(['utime', 'cminflt'], axis=1, inplace=True)
print(f"Remaining features ({len(df2_features_n.columns)}): {df2_features_n.columns.tolist()}")

### 3.4 Group Analysis by Application

In [ ]:
# SOLUTION: Recombine features with labels for group analysis
df2_with_label = df2_features_n.copy()
df2_with_label['ApplicationName'] = df2_labels

print("Descriptive Statistics by Application:")
print("="*80)
for col in df2_features_n.columns:
    print(f"\n{col}:")
    print(df2_with_label.groupby('ApplicationName')[col].describe())

In [ ]:
# SOLUTION: Analyze feature discriminability
print("\nFeature Discriminability Analysis:")
print("="*80)
for col in df2_features_n.columns:
    stats = df2_with_label.groupby('ApplicationName')[col].describe()
    fb_mean = stats.loc['Facebook', 'mean']
    wa_mean = stats.loc['WhatsApp', 'mean']
    fb_std = stats.loc['Facebook', 'std']
    wa_std = stats.loc['WhatsApp', 'std']
    
    # Calculate separation metric
    separation = abs(fb_mean - wa_mean) / (fb_std + wa_std)
    print(f"\n{col}:")
    print(f"  Facebook mean: {fb_mean:8.4f} (std: {fb_std:8.4f})")
    print(f"  WhatsApp mean: {wa_mean:8.4f} (std: {wa_std:8.4f})")
    print(f"  Separation metric: {separation:8.4f} (higher = better discriminator)")

In [ ]:
# SOLUTION: Remove features with poor discriminability
print("\nRemoving features with poor discriminability: CPU_USAGE, lru")
df2_features_n.drop(['CPU_USAGE', 'lru'], axis=1, inplace=True)
print(f"\nFinal feature set ({len(df2_features_n.columns)}): {df2_features_n.columns.tolist()}")

### 3.5 Feature Selection Summary

In [ ]:
# SOLUTION: Save the selected features
df2_features_n1 = df2_features_n_backup[['cutime', 'num_threads', 'otherPrivateDirty', 'priority']]

# Convert labels to numeric
labels_numeric = df2_labels.replace(['Facebook', 'WhatsApp'], [0, 1])

# Save to CSV
labels_numeric.to_csv('sherlock_2apps_labels.csv', header=True, index=False)
df2_features_n1.to_csv('sherlock_2apps_features.csv', index=False)

print("Selected Features:")
print(df2_features_n1.head(10))
print(f"\nShape: {df2_features_n1.shape}")
print(f"\nFeatures saved to:")
print("  - sherlock_2apps_features.csv")
print("  - sherlock_2apps_labels.csv")

## 4. Training and Validating with Selected Features

In [ ]:
# SOLUTION: Prepare data for training
features = df2_features_n1.copy()
labels = df2_labels.copy()

# Train-test split
train_F, test_F, train_L, test_L = train_test_split(features, labels, test_size=0.2, random_state=42)

print(f"Training set: {train_F.shape}")
print(f"Test set: {test_F.shape}")
print(f"\nFeatures used: {features.columns.tolist()}")

In [ ]:
# SOLUTION: Train Logistic Regression model
model_lr = LogisticRegression(solver='lbfgs', max_iter=1000)
%time model_lr.fit(train_F, train_L)

print("\n" + "="*60)
print("LOGISTIC REGRESSION - SELECTED FEATURES")
print("="*60)
acc_lr = model_evaluate(model_lr, test_F, test_L)
print(f"\nAccuracy: {acc_lr:.4f}")

In [ ]:
# SOLUTION: Train Decision Tree model
model_dtc = DecisionTreeClassifier(criterion='entropy', max_depth=3, min_samples_split=8, random_state=42)
%time model_dtc.fit(train_F, train_L)

print("\n" + "="*60)
print("DECISION TREE - SELECTED FEATURES")
print("="*60)
acc_dtc = model_evaluate(model_dtc, test_F, test_L)
print(f"\nAccuracy: {acc_dtc:.4f}")

## 5. Better Validation: K-Fold Cross-Validation

In [ ]:
# SOLUTION: Implement k-fold cross-validation
from sklearn.model_selection import KFold, cross_val_score

kfold = KFold(n_splits=10, shuffle=True, random_state=42)

# Logistic Regression with k-fold
model_lr_kfold = LogisticRegression(solver='lbfgs', max_iter=1000)
results_lr_kfold = cross_val_score(model_lr_kfold, train_F, train_L, cv=kfold)

print("\n" + "="*60)
print("K-FOLD CROSS-VALIDATION RESULTS (k=10)")
print("="*60)
print("\nLogistic Regression:")
print(f"  Individual fold accuracies: {results_lr_kfold}")
print(f"  Mean Accuracy: {results_lr_kfold.mean():.4f}")
print(f"  Std Deviation: {results_lr_kfold.std():.4f}")
print(f"  95% Confidence Interval: [{results_lr_kfold.mean() - 1.96*results_lr_kfold.std():.4f}, {results_lr_kfold.mean() + 1.96*results_lr_kfold.std():.4f}]")

In [ ]:
# SOLUTION: Decision Tree with k-fold
model_dtc_kfold = DecisionTreeClassifier(criterion='entropy', max_depth=3, min_samples_split=8, random_state=42)
results_dtc_kfold = cross_val_score(model_dtc_kfold, train_F, train_L, cv=kfold)

print("\nDecision Tree:")
print(f"  Individual fold accuracies: {results_dtc_kfold}")
print(f"  Mean Accuracy: {results_dtc_kfold.mean():.4f}")
print(f"  Std Deviation: {results_dtc_kfold.std():.4f}")
print(f"  95% Confidence Interval: [{results_dtc_kfold.mean() - 1.96*results_dtc_kfold.std():.4f}, {results_dtc_kfold.mean() + 1.96*results_dtc_kfold.std():.4f}]")

## 6. Results Comparison and Summary

In [ ]:
# SOLUTION: Create comprehensive results summary
results_summary = pd.DataFrame({
    'Model': ['Logistic Regression', 'Decision Tree'],
    'Test Accuracy': [acc_lr, acc_dtc],
    'K-Fold Mean': [results_lr_kfold.mean(), results_dtc_kfold.mean()],
    'K-Fold Std': [results_lr_kfold.std(), results_dtc_kfold.std()]
})

print("\n" + "="*80)
print("COMPREHENSIVE RESULTS SUMMARY")
print("="*80)
print(results_summary.to_string(index=False))
print("="*80)

In [ ]:
# SOLUTION: Visualize k-fold results
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: K-fold accuracies
folds = np.arange(1, 11)
axes[0].plot(folds, results_lr_kfold, 'o-', label='Logistic Regression', linewidth=2, markersize=8)
axes[0].plot(folds, results_dtc_kfold, 's-', label='Decision Tree', linewidth=2, markersize=8)
axes[0].axhline(y=results_lr_kfold.mean(), color='blue', linestyle='--', alpha=0.5, label=f'LR Mean: {results_lr_kfold.mean():.4f}')
axes[0].axhline(y=results_dtc_kfold.mean(), color='orange', linestyle='--', alpha=0.5, label=f'DTC Mean: {results_dtc_kfold.mean():.4f}')
axes[0].set_xlabel('Fold Number', fontsize=12)
axes[0].set_ylabel('Accuracy', fontsize=12)
axes[0].set_title('K-Fold Cross-Validation Accuracies', fontsize=13, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)
axes[0].set_ylim([0.5, 1.0])

# Plot 2: Mean accuracy with error bars
models = ['Logistic\nRegression', 'Decision\nTree']
means = [results_lr_kfold.mean(), results_dtc_kfold.mean()]
stds = [results_lr_kfold.std(), results_dtc_kfold.std()]
colors = ['steelblue', 'coral']

bars = axes[1].bar(models, means, yerr=stds, capsize=10, alpha=0.7, color=colors, edgecolor='black', linewidth=2)
axes[1].set_ylabel('Accuracy', fontsize=12)
axes[1].set_title('Mean Accuracy with Standard Deviation', fontsize=13, fontweight='bold')
axes[1].set_ylim([0.5, 1.0])
axes[1].grid(axis='y', alpha=0.3)

# Add value labels on bars
for i, (bar, mean, std) in enumerate(zip(bars, means, stds)):
    axes[1].text(bar.get_x() + bar.get_width()/2, mean + std + 0.02,
                f'{mean:.4f}\n±{std:.4f}', ha='center', va='bottom', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.show()

## 7. Key Findings and Discussion

### Feature Selection Process:
1. **Correlation Analysis**: Identified and removed highly correlated features (vsize, queue, guest_time, utime, cminflt)
2. **Discriminability Analysis**: Removed features with poor separation between classes (CPU_USAGE, lru)
3. **Final Feature Set**: cutime, num_threads, otherPrivateDirty, priority

### Model Performance:
- **Logistic Regression**: Consistent performance across folds
- **Decision Tree**: Slightly better performance with lower variance

### Validation Strategy:
- **K-Fold Cross-Validation**: Provides more robust estimate of model performance
- **Uncertainty Quantification**: Standard deviation shows model stability

### Why Feature Selection Matters:
1. **Reduces Overfitting**: Fewer features = simpler model
2. **Improves Interpretability**: Easier to understand which features drive predictions
3. **Reduces Computational Cost**: Faster training and inference
4. **Removes Noise**: Irrelevant features can degrade performance

### Cybersecurity Application:
This approach can be used for:
- **Malware Detection**: Identify malicious apps based on resource usage patterns
- **Behavioral Analysis**: Detect anomalous application behavior
- **Real-time Monitoring**: Classify running applications efficiently
- **Threat Intelligence**: Build profiles of known malicious applications